In [ ]:
!pip install -U tensorflow-text

In [ ]:
!wget 

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import tensorflow_text
from google.colab import files
import pandas as pd

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prfs

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('drive/MyDrive/data.csv', sep=';')
text_var = 'text'
class_var = 'mark'

In [ ]:
data.head() 

In [ ]:
df = data[class_var].value_counts(normalize=True) * 100
df.plot.bar(x=class_var)

In [ ]:
data = data[(data[class_var] != -1)]

In [ ]:
marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

In [ ]:
import gensim
import pymorphy2
import pprint
from collections import defaultdict
from gensim import corpora
from gensim import models
import re

In [ ]:
morph = pymorphy2.MorphAnalyzer()


In [1]:
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
remove_stopwords('В суде США начинается процесс против россиянина, рассылавшего спам.')

'В суде США начинается процесс против россиянина, рассылавшего спам.'

In [ ]:
train_vol = 0.7

In [ ]:
train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [ ]:
check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

In [ ]:
train_embeddings = []
for text in train_data:
    embedding = embed(str(text))[0]
    train_embeddings.append(embedding)

In [ ]:
test_embeddings = []
for text in test[text_var]:
    embedding = embed(str(text))[0]
    test_embeddings.append(embedding)

In [ ]:
# nbg = GaussianNB()
# nbb = BernoulliNB()
# lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [ ]:
classes = train[class_var].tolist()
classes_to_check = test[class_var].tolist()

In [ ]:
# nbg.fit(train_embeddings,classes)
# nbb.fit(train_embeddings,classes)
# lg.fit(train_embeddings,classes)
svm.fit(train_embeddings,classes)

In [ ]:
# classesNBG = nbg.predict(test_embeddings)
# classesNBB = nbb.predict(test_embeddings)
# classesLG = lg.predict(test_embeddings)
classesSVM = svm.predict(test_embeddings)

In [ ]:
metrics = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')

In [ ]:
metrics

In [ ]:
df_results = pd.DataFrame(columns=['classifier', 'train_size', 'average_precision',
                                   'average_recall', 'average_fscore'])
precision_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
recall_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
fscore_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}

data = pd.read_excel('/opt/projects/docs_to_vecs/sample_data/mcko_random_sample_C1_9602.xlsx')

text_var = 'info_new'
class_var = 'mark'

data = data.loc[(data[class_var] != -1)]

marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new

train_vol = 0.7

for i in range(10):
    train, tmp = train_test_split(data, train_size = train_vol, stratify = data[class_var])
    train = train.drop_duplicates(subset=[text_var])
    test = data
    check = []
    train_data = train[text_var].tolist()
    for text in data[text_var]:
        if text not in train_data:
            check.append('KEEP')
        else:
            check.append('IGNORE')
    test['check'] = check
    test = test.loc[test['check'] == 'KEEP']

    train_embeddings = []
    for text in train_data:
        embedding = embed(str(text))[0]
        train_embeddings.append(embedding)
    print('Прогон {}, вектора для обучения готовы.'.format(i+1))

    test_embeddings = []
    for text in test[text_var]:
        embedding = embed(str(text))[0]
        test_embeddings.append(embedding)
    print('Прогон {}, вектора для тестирования готовы.'.format(i+1))

    nbg = GaussianNB()
    nbb = BernoulliNB()
    svm = SVC(kernel='poly', gamma='scale', probability=True)
    lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)

    classes = train[class_var].tolist()

    nbg.fit(train_embeddings,classes)
    nbb.fit(train_embeddings,classes)
    svm.fit(train_embeddings,classes)
    lg.fit(train_embeddings,classes)
    print('Прогон {}, все модели обучены.'.format(i+1))

    target = 0

    classesNBG = nbg.predict(test_embeddings)
    scoresNBG = [score[target] for score in nbg.predict_proba(test_embeddings)]
    classesNBB = nbb.predict(test_embeddings)
    scoresNBB = [score[target] for score in nbb.predict_proba(test_embeddings)]
    classesSVM = svm.predict(test_embeddings)
    scoresSVM = [score[target] for score in svm.predict_proba(test_embeddings)]
    classesLG = lg.predict(test_embeddings)
    scoresLG = [score[target] for score in lg.predict_proba(test_embeddings)]

    classes_to_check = test[class_var].tolist()

    precisionNBG, recallNBG, fscoreNBG = prfs(classes_to_check, classesNBG, pos_label = 0, average = 'binary')[0:3]
    precisionNBB, recallNBB, fscoreNBB = prfs(classes_to_check, classesNBB, pos_label = 0, average = 'binary')[0:3]
    precisionSVM, recallSVM, fscoreSVM = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')[0:3]
    precisionLG, recallLG, fscoreLG = prfs(classes_to_check, classesLG, pos_label = 0, average = 'binary')[0:3]

    precision_all['NBG'].append(precisionNBG)
    precision_all['NBB'].append(precisionNBB)
    precision_all['SVM'].append(precisionSVM)
    precision_all['LG'].append(precisionLG)
    recall_all['NBG'].append(recallNBG)
    recall_all['NBB'].append(recallNBB)
    recall_all['SVM'].append(recallSVM)
    recall_all['LG'].append(recallLG)
    fscore_all['NBG'].append(fscoreNBG)
    fscore_all['NBB'].append(fscoreNBB)
    fscore_all['SVM'].append(fscoreSVM)
    fscore_all['LG'].append(fscoreLG)
    print('Прогон {} закончен.'.format(i+1))


for model in ['NBG', 'NBB', 'SVM', 'LG']:
    df_results = df_results.append({'classifier': model,
                                'average_precision':sum(precision_all[model])/10,
                                'average_recall':sum(recall_all[model])/10,
                                'average_fscore':sum(fscore_all[model])/10},
                                 ignore_index=True)

df_results.set_index('classifier')
df_results_t = df_results.transpose()
df_results_t.columns = df_results_t.iloc[0]
df_results_t.reset_index(inplace=True)
df_results_t = df_results_t.drop([0], axis=0)
df_results_t.plot.bar('index')